In [1]:
# ! pip install micrograd

## Alternate implementation 
https://sidsite.com/posts/autodiff/

In [1]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        import pdb;pdb.set_trace()
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [2]:

a = Value(-4.0)
b = Value(2.0)
c = a + b
# d = a * b + b**3
# c += c + 1
# c += 1 + c + (-a)
# d += d * 2 + (b + a).relu()
# d += 3 * d + (b - a).relu()
# e = c - d
# f = e**2
# g = f / 2.0
# g += 10.0 / f
# print(f'{g.data:.4f}') # prints 24.7041, the outcome of this forward pass
# g.backward()
# print(f'{a.grad:.4f}') # prints 138.8338, i.e. the numerical value of dg/da
# print(f'{b.grad:.4f}') # prints 645.5773, i.e. the numerical value of dg/db


In [3]:
c.backward()

> <ipython-input-1-64348745a563>(69)backward()
     67         self.grad = 1
     68         import pdb;pdb.set_trace()
---> 69         for v in reversed(topo):
     70             v._backward()
     71 

ipdb> p topo 
[Value(data=-4.0, grad=0), Value(data=2.0, grad=0), Value(data=-2.0, grad=1)]
ipdb> n
> <ipython-input-1-64348745a563>(70)backward()
     68         import pdb;pdb.set_trace()
     69         for v in reversed(topo):
---> 70             v._backward()
     71 
     72     def __neg__(self): # -self

ipdb> s
--Call--
> <ipython-input-1-64348745a563>(16)_backward()
     14         out = Value(self.data + other.data, (self, other), '+')
     15 
---> 16         def _backward():
     17             self.grad += out.grad
     18             other.grad += out.grad

ipdb> p self
Value(data=-4.0, grad=0)
ipdb> p out 
Value(data=-2.0, grad=1)
ipdb> p out.grad
1
ipdb> p self.grad
0
ipdb> q


BdbQuit: 

In [6]:
c.backward()

> <ipython-input-4-64348745a563>(69)backward()
     67         self.grad = 1
     68         import pdb;pdb.set_trace()
---> 69         for v in reversed(topo):
     70             v._backward()
     71 

ipdb> p self.grad
1
ipdb> p topo[-1]
Value(data=-2.0, grad=1)
ipdb> n
> <ipython-input-4-64348745a563>(70)backward()
     68         import pdb;pdb.set_trace()
     69         for v in reversed(topo):
---> 70             v._backward()
     71 
     72     def __neg__(self): # -self

ipdb> s
--Call--
> <ipython-input-4-64348745a563>(16)_backward()
     14         out = Value(self.data + other.data, (self, other), '+')
     15 
---> 16         def _backward():
     17             self.grad += out.grad
     18             other.grad += out.grad

ipdb> p self
Value(data=-4.0, grad=0)
ipdb> p out 
Value(data=-2.0, grad=1)
ipdb> p out.grad 
1
ipdb> c


In [15]:
a.grad

1

In [20]:
a._backward()

## Example From PyTorch 
https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd